# Supervised Learning   

## League of Legends SoloQ matches at 15 minutes 2024

### Introduction

#### Dataset

This dataset was obtained from [Kaggle's LOL page](https://www.kaggle.com/datasets/karlorusovan/league-of-legends-soloq-matches-at-10-minutes-2024/data). The dataset contains data about the first 15 minutes of gameplay for over 24 thousand solo queue matches taken from european servers (EUNE and EUW).

Average ELO of the matches is between mid emerald to high diamond, which represents the top 10% of players, therefore (hopefully) reducing the probability of a player doing a game-changing mistake.

There are 14 features for red, and 14 features for the blue team (feature blueTeamFirstBlood counts for both teams since it tells us which team got the first kill), with the target feature being blueWin. blueWin == 1 indicates a victory of the blue team, and blueWin == 0 means that the red team won.

#### Problem

The main purpose of the dataset is to help train models to predict the winner based on how the first 15 minutes of the match played out.

#### Solution

We will train a supervised learning model using the aforementioned dataset. Therefore, the downloaded dataset will be split into a training and a test set, so that we can train the model and evaluate it afterwards. We will evaluate the model utilizing accuracy, precision, sensitivity and F-measure, depending on our needs.

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

# Load the data
data = pd.read_csv('match_data_v5.csv')

data.columns =["matchID","blueTeamControlWardsPlaced","blueTeamWardsPlaced","blueTeamTotalKills","blueTeamDragonKills","blueTeamHeraldKills","blueTeamTowersDestroyed","blueTeamInhibitorsDestroyed","blueTeamTurretPlatesDestroyed","blueTeamFirstBlood","blueTeamMinionsKilled","blueTeamJungleMinions","blueTeamTotalGold","blueTeamXp","blueTeamTotalDamageToChamps","redTeamControlWardsPlaced","redTeamWardsPlaced","redTeamTotalKills","redTeamDragonKills","redTeamHeraldKills","redTeamTowersDestroyed","redTeamInhibitorsDestroyed","redTeamTurretPlatesDestroyed","redTeamMinionsKilled","redTeamJungleMinions","redTeamTotalGold","redTeamXp","redTeamTotalDamageToChamps","blueWin","empty"]
data = data.drop(columns=['matchID'])
data = data.drop(columns=['empty'])

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

all_inputs = data.drop('blueWin', axis=1)
all_labels = data['blueWin'].values

(training_inputs,
 testing_inputs,
 training_classes,
 testing_classes) = train_test_split(all_inputs, all_labels, test_size=0.10)

print(data["blueWin"].value_counts())
print(data["blueWin"].value_counts(normalize=True))

# Train the model decision tree
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(
    max_depth=3

)

import sklearn.tree as tree

model.fit(training_inputs, training_classes)

print("Decision Tree")

file = open('iris_dtc.dot', 'w')
out_file = tree.export_graphviz(model, out_file=file)
file.close()

'''
model.fit(training_inputs, training_classes)

print("Decision Tree")
print(model.score(testing_inputs, testing_classes))

# Train the model SVM
from sklearn.svm import SVC
model = SVC()
model.fit(training_inputs, training_classes)

print("SVM")
print(model.score(testing_inputs, testing_classes))

# Train the model KNN

from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()

model.fit(training_inputs, training_classes)

print("KNN")
print(model.score(testing_inputs, testing_classes))

'''

from sklearn.model_selection import cross_val_score

decision_tree_classifier = DecisionTreeClassifier(
    max_depth=5,

)

# cross_val_score returns a list of the scores, which we can visualize
# to get a reasonable estimate of our classifier's performance
cv_scores = cross_val_score(decision_tree_classifier, all_inputs, all_labels, cv=10)
plt.hist(cv_scores)
plt.title('Average score: {}'.format(np.mean(cv_scores)))



blueWin
0    12245
1    11980
Name: count, dtype: int64
blueWin
0    0.50547
1    0.49453
Name: proportion, dtype: float64
Decision Tree


InvocationException: GraphViz's executables not found